# Hand digit classifier 
---
## Incremental network quantization

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from skimage import io

In [85]:
def getBounderyExponents(W, b):
    s = torch.max(torch.abs(W)).item()
    n1 = np.floor(np.log2(4*(s/3)))
    n2 = n1 + 1 - (2**(b - 1))/2
    return n1, n2

def getQuantizationMask(W, percentage, T):
    w = W.view(-1)
    t = T.view(-1)
    idx = t == 1

    numberOfWeights = w.size(dim=0)
    numberOfQWeights = int(percentage*numberOfWeights - t[idx].size(dim=0))

    t_aux = torch.Tensor(np.ones_like(T)).view(-1)
    w = w*(t_aux - t)
    w = torch.abs(w)
    sorted_w, indices_w = w.sort()
    t[indices_w[-numberOfQWeights:]] = 1
    
    return t.view(T.size())

def quantizeWeights(W, T):
    T_aux = torch.Tensor(np.ones_like(T))
    W1 = W*(T_aux - T)
    W2 = W*T
    closestExp = np.floor(np.log2(np.abs(W2*4/3)))
    return torch.abs(closestExp), W1 + 2**closestExp

In [ ]:
W = torch.tensor(np.array([
    [0.01, 0.02, -0.2, 0.04, 0.33],
    [0.17, -0.42, -0.33, 0.02, -0.05], 
    [0.02, 0.83, -0.03, 0.03, 0.06],
    [-0.9, 0.07, 0.11, 0.87, -0.36], 
    [-0.73, 0.41, 0.42, 0.39, 0.47]]))
bit_length = 4
n1, n2 = getBounderyExponents(W, bit_length)
print(n1, n2)
T = torch.Tensor(np.zeros_like(W))
T = getQuantizationMask(W, 0.5, T)
print(W)
print(T)
T = getQuantizationMask(W, 0.75, T)
print(W)
print(T)
T = getQuantizationMask(W, 1, T)
print(W)
print(T)

In [ ]:
weight = np.array([-0.42, 0.87])

closestExp = np.floor(np.log2(np.abs(weight*4/3)))
print("{} -> {}*2^{}".format(weight, np.sign(weight), closestExp))

In [88]:
W = torch.tensor(np.array([
    [0.01, 0.02, -0.2, 0.04, 0.33],
    [0.17, -0.42, -0.33, 0.02, -0.05], 
    [0.02, 0.83, -0.03, 0.03, 0.06],
    [-0.9, 0.07, 0.11, 0.87, -0.36], 
    [-0.73, 0.41, 0.42, 0.39, 0.47]]))
    
T = torch.Tensor(np.zeros_like(W))
T = getQuantizationMask(W, 0.5, T)
print(W)
print(T)
quantizeWeights(W, T)

tensor([[ 0.0100,  0.0200, -0.2000,  0.0400,  0.3300],
        [ 0.1700, -0.4200, -0.3300,  0.0200, -0.0500],
        [ 0.0200,  0.8300, -0.0300,  0.0300,  0.0600],
        [-0.9000,  0.0700,  0.1100,  0.8700, -0.3600],
        [-0.7300,  0.4100,  0.4200,  0.3900,  0.4700]], dtype=torch.float64)
tensor([[0., 0., 0., 0., 1.],
        [0., 1., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 1., 1.],
        [1., 1., 1., 1., 1.]])


C:\Users\iusti\AppData\Local\Temp\ipykernel_10572\4171450455.py:27: RuntimeWarning: divide by zero encountered in log2
  closestExp = np.floor(np.log2(np.abs(W2*4/3)))


(tensor([[-inf, -inf, -inf, -inf, -2.],
         [-inf, -1., -2., -inf, -inf],
         [-inf,  0., -inf, -inf, -inf],
         [ 0., -inf, -inf,  0., -2.],
         [-1., -1., -1., -1., -1.]], dtype=torch.float64),
 tensor([[ 0.0100,  0.0200, -0.2000,  0.0400,  0.2500],
         [ 0.1700,  0.5000,  0.2500,  0.0200, -0.0500],
         [ 0.0200,  1.0000, -0.0300,  0.0300,  0.0600],
         [ 1.0000,  0.0700,  0.1100,  1.0000,  0.2500],
         [ 0.5000,  0.5000,  0.5000,  0.5000,  0.5000]], dtype=torch.float64))